## Importing libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import gdown
import keras
import pandas as pd
import pathlib

from pathlib import Path

## Connecting to google drive disk to extract data
In my disk I have two folders with train and test data.
In every folder there are pictures with people having nine different emotions:
* neutral 
* anger 
* contempt 
* disgust 
* fear 
* happy 
* sad 
* surprise 
* uncertain

In [ ]:
url = 'https://drive.google.com/uc?export=download&id=12QrDrLT1F-X7UycvOoApXFqxTw3Zx93K'
output = 'test_kaggle.zip'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=12QrDrLT1F-X7UycvOoApXFqxTw3Zx93K
To: /content/test_kaggle.zip
100%|██████████| 222M/222M [00:05<00:00, 41.6MB/s]


'test_kaggle.zip'

In [ ]:
url = 'https://drive.google.com/uc?export=download&id=1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16'
output = 'train.zip'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16
To: /content/train.zip
100%|██████████| 2.28G/2.28G [00:50<00:00, 44.8MB/s]


'train.zip'

In [ ]:
! unzip train.zip
! unzip test_kaggle.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: test_kaggle/0.jpg       
  inflating: test_kaggle/1.jpg       
  inflating: test_kaggle/10.jpg      
  inflating: test_kaggle/100.jpg     
  inflating: test_kaggle/1000.jpg    
  inflating: test_kaggle/1001.jpg    
  inflating: test_kaggle/1002.jpg    
  inflating: test_kaggle/1003.jpg    
  inflating: test_kaggle/1004.jpg    
  inflating: test_kaggle/1005.jpg    
  inflating: test_kaggle/1006.jpg    
  inflating: test_kaggle/1007.jpg    
  inflating: test_kaggle/1008.jpg    
  inflating: test_kaggle/1009.jpg    
  inflating: test_kaggle/101.jpg     
  inflating: test_kaggle/1010.jpg    
  inflating: test_kaggle/1011.jpg    
  inflating: test_kaggle/1012.jpg    
  inflating: test_kaggle/1013.jpg    
  inflating: test_kaggle/1014.jpg    
  inflating: test_kaggle/1015.jpg    
  inflating: test_kaggle/1016.jpg    
  inflating: test_kaggle/1017.jpg    
  inflating: test_kaggle/1018.jpg    
  inflating: test_ka

In [ ]:
df = pd.read_csv('train.csv', index_col=0)


In [ ]:
df.head()

,image_path,emotion
0,./train/anger/0.jpg,anger
1,./train/anger/1.jpg,anger
2,./train/anger/10.jpg,anger
3,./train/anger/100.jpg,anger
4,./train/anger/1000.jpg,anger


## Creating directories to folder's data

In [ ]:
base_dir = Path('./train/')

In [ ]:
base_dir_test = Path('./test_kaggle/')

In [ ]:
pic_num = []

for i_elem in range(5000):
  pic_num.append('{}.jpg'.format(i_elem))

df_test = pd.DataFrame(pic_num, columns=['img_name'])

In [ ]:
df_test.tail()

,img_name
4995,4995.jpg
4996,4996.jpg
4997,4997.jpg
4998,4998.jpg
4999,4999.jpg


## Choosing neural model and creating ImageDataGenerators for extracting pictures for the model

### Implementation of DenseNet neural model

In [ ]:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input as preprocess_input_dense

In [ ]:
BATCH_SIZE = 64
IMAGE_SIZE = 224

datagen_aug = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input_dense,
                                                              validation_split=0.25,
                                                              rotation_range=40,
                                                              width_shift_range=0.2,
                                                              height_shift_range=0.2,
                                                              zoom_range=0.2,
                                                              horizontal_flip=True,
                                                              fill_mode='nearest')
datagen_aug_test = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input_dense)

train_generator = datagen_aug.flow_from_directory(batch_size=BATCH_SIZE,
                                              directory=base_dir,
                                              shuffle=True,
                                              subset='training',
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                              class_mode='categorical',
                                              seed=42)

val_generator = datagen_aug.flow_from_directory(batch_size=BATCH_SIZE,
                                            directory=base_dir, 
                                            shuffle=True,
                                            subset='validation',
                                            target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                            class_mode='categorical',
                                            seed=42)

test_generator = datagen_aug_test.flow_from_dataframe(dataframe=df_test,
                                              x_col='img_name',
                                              y_col=None,
                                              directory=base_dir_test,
                                              class_mode=None,
                                              batch_size=BATCH_SIZE,
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              shuffle=False,
                                              seed=42)


Found 37540 images belonging to 9 classes.
Found 12507 images belonging to 9 classes.
Found 5000 validated image filenames.


## Looking at the structure of our model
The DenseNet201 neural model has weights from 'imagenet', but we would like it to be able recognizing emotions. That is why we set 'Trainable params' as zero to enable our model to learn patterns from our images in the future.

In [ ]:
IMAGE_SIZE = (224, 224, 3)

base_model = DenseNet201(weights='imagenet',
                      include_top=False,
                      input_shape=IMAGE_SIZE)
base_model.trainable = False
base_model.summary()

74836368/74836368 [==============================] - 5s 0us/step
Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                       

In [ ]:
base_model.trainable = True

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(9, activation='softmax')
])


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
print("Quantity of layers in base model: {}".format(len(base_model.layers)))

Quantity of layers in base model: 707


### Choosing optimal quantity of layers

In [ ]:
fine_tune_at = 400

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                                 
 global_average_pooling2d (G  (None, 1920)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 9)                 17289     
                                                                 
Total params: 18,339,273
Trainable params: 11,305,225
Non-trainable params: 7,034,048
_________________________________________________________________


### At this step we train the model

In [ ]:
EPOCHS=10
history = model.fit_generator(train_generator,
                              epochs=EPOCHS,
                              validation_data=val_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/10
587/587 [==============================] - 927s 2s/step - loss: 1.9899 - accuracy: 0.2759 - val_loss: 1.7621 - val_accuracy: 0.3588
Epoch 2/10
587/587 [==============================] - 870s 1s/step - loss: 1.6671 - accuracy: 0.3986 - val_loss: 1.6209 - val_accuracy: 0.4078
Epoch 3/10
587/587 [==============================] - 871s 1s/step - loss: 1.5492 - accuracy: 0.4386 - val_loss: 1.5512 - val_accuracy: 0.4356
Epoch 4/10
587/587 [==============================] - 867s 1s/step - loss: 1.4792 - accuracy: 0.4678 - val_loss: 1.5052 - val_accuracy: 0.4505
Epoch 5/10
587/587 [==============================] - 871s 1s/step - loss: 1.4229 - accuracy: 0.4855 - val_loss: 1.4857 - val_accuracy: 0.4567
Epoch 6/10
587/587 [==============================] - 866s 1s/step - loss: 1.3802 - accuracy: 0.5019 - val_loss: 1.4571 - val_accuracy: 0.4695
Epoch 7/10
587/587 [==============================] - 869s 1s/step - loss: 1.3403 - accuracy: 0.5170 - val_loss: 1.4468 - val_accuracy: 0.4728

## Saving our trained model as *.pb file for future implementation using OpenCV

In [ ]:
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [ ]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="frozen_graph.pb",
                  as_text=False)

--------------------------------------------------
Frozen model layers: 
x
sequential/densenet201/zero_padding2d/Pad/paddings
sequential/densenet201/zero_padding2d/Pad
sequential/densenet201/conv1/conv/Conv2D/ReadVariableOp/resource
sequential/densenet201/conv1/conv/Conv2D/ReadVariableOp
sequential/densenet201/conv1/conv/Conv2D
sequential/densenet201/conv1/bn/ReadVariableOp/resource
sequential/densenet201/conv1/bn/ReadVariableOp
sequential/densenet201/conv1/bn/ReadVariableOp_1/resource
sequential/densenet201/conv1/bn/ReadVariableOp_1
sequential/densenet201/conv1/bn/FusedBatchNormV3/ReadVariableOp/resource
sequential/densenet201/conv1/bn/FusedBatchNormV3/ReadVariableOp
sequential/densenet201/conv1/bn/FusedBatchNormV3/ReadVariableOp_1/resource
sequential/densenet201/conv1/bn/FusedBatchNormV3/ReadVariableOp_1
sequential/densenet201/conv1/bn/FusedBatchNormV3
sequential/densenet201/conv1/relu/Relu
sequential/densenet201/zero_padding2d_1/Pad/paddings
sequential/densenet201/zero_padding2d_1/P

'./frozen_models/frozen_graph.pb'